## 🌟 Hands-on Practice with Function Call

Next, we'll implement a standalone function call example using the Yi-1.5-9B-Chat model. This example doesn't rely on any specific framework but directly uses the Hugging Face transformers library to load and use the model.

First, let's install the necessary dependencies:
⚠️ Please be mindful of your computer's GPU memory here

In [ ]:
!pip install transformers torch

Now let's start building our function call implementation step by step.
Just like before, we'll use addition, subtraction, and multiplication functions as examples.
#### Step 1: Import necessary libraries and define functions

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Define available functions
def multiply(a: int, b: int) -> int:
    return a * b

def plus(a: int, b: int) -> int:
    return a + b

def minus(a: int, b: int) -> int:
    return a - b
# You can add your own functions here if needed
# Function mapping
available_functions = {
    "multiply": multiply,
    "plus": plus,
    "minus": minus
}

#### Step 2: Load the Yi model and tokenizer (we'll use transformers for this step)

In [ ]:
# Load Yi model and tokenizer
model_path = "01-ai/Yi-1.5-9B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")

#### Step 3: Implement functionality to generate responses and parse function calls

In [ ]:
# Generate Yi's response
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, temperature=0.7, top_p=0.95)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Human:")[0].strip()

# Parse output, extract function information
def parse_function_call(response):
    try:
        # Try to extract JSON-formatted function call from the response
        start = response.index("{")
        end = response.rindex("}") + 1
        function_call_json = response[start:end]
        function_call = json.loads(function_call_json)
        return function_call
    except (ValueError, json.JSONDecodeError):
        return None

# Execute function call (bridge for function information transfer)
def execute_function(function_name: str, arguments: dict) -> Any:
    if function_name in available_functions:
        return available_functions[function_name](**arguments)
    else:
        raise ValueError(f"Function {function_name} not found")

#### Step 4: Implement the main loop

In [ ]:
# Main loop
def main():
    # This prompt template contains function information. If you need to add other functions, you should synchronize them with the large model here as well
    system_prompt = """You are an AI assistant capable of calling functions to perform tasks. When a user asks a question that requires calling a function, respond with a JSON object containing the function name and arguments. Available functions are:
    - multiply(a: int, b: int) -> int: Multiplies two integers
    - plus(a: int, b: int) -> int: Adds two integers
    - minus(a: int, b: int) -> int: Subtracts two integers
    For example, if the user asks "What is 5 plus 3?", respond with:
    {"function": "plus", "arguments": {"a": 5, "b": 3}}
    If no function call is needed, respond normally."""

    conversation_history = [f"System: {system_prompt}"]

    while True:
        user_input = input("Human: ")
        if user_input.lower() == 'exit':
            break

        # Add user input to conversation history
        conversation_history.append(f"Human: {user_input}")
        
        # Build complete prompt
        full_prompt = "\n".join(conversation_history) + "\nAssistant:"

        # Get model response
        response = generate_response(full_prompt)
        print(f"Model response: {response}")

        # Parse possible function call
        function_call = parse_function_call(response)

        if function_call:
            function_name = function_call["function"]
            arguments = function_call["arguments"]

            try:
                # Execute function
                result = execute_function(function_name, arguments)

                # Add result to conversation history
                conversation_history.append(f"Assistant: The result of {function_name}({arguments}) is {result}")
                print(f"Assistant: The result of {function_name}({arguments}) is {result}")
            except Exception as e:
                error_message = f"An error occurred: {str(e)}"
                conversation_history.append(f"Assistant: {error_message}")
                print(f"Assistant: {error_message}")
        else:
            # If no function call, directly add model's response to conversation history
            conversation_history.append(f"Assistant: {response}")
            print(f"Assistant: {response}")

if __name__ == "__main__":
    main()

#### Running the Example

To run this example, you need to ensure that you've downloaded the Yi-1.5-9B-Chat model, or change the `model_path` to the actual path of the model. Then, you can directly run this Python script.

Usage example:

In [ ]:
Human: What is 5 plus 3?
Model response: Here's the function call to perform the addition:
{"function": "plus", "arguments": {"a": 5, "b": 3}}
Assistant: The result of plus({'a': 5, 'b': 3}) is 8